In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import time

from model import Generator, Discriminator
from config import irgan_config
from data_utils import RecDataset, DataProvider
from evaluation.rec_evaluator import RecEvaluator

In [2]:
epochs = irgan_config.epochs
batch_size = irgan_config.batch_size
epochs_d = irgan_config.epochs_d
epochs_g = irgan_config.epochs_g
emb_dim = irgan_config.emb_dim
eta_G = irgan_config.eta_G
eta_D = irgan_config.eta_D
device = irgan_config.device
weight_decay_g = irgan_config.weight_decay_g
weight_decay_d = irgan_config.weight_decay_d

In [3]:
# Hyper-parameters and datset-specific parameters
rec_dataset = RecDataset(irgan_config.dir_path)
all_users = rec_dataset.get_users()
all_items = rec_dataset.get_items()
num_users = rec_dataset.get_num_users()
num_items = rec_dataset.get_num_items()
bought_mask = rec_dataset.get_bought_mask().to(device)
eval_dict = rec_dataset.get_interaction_records("test")
train_ui = rec_dataset.get_user_item_pairs()

In [4]:
dp = DataProvider(device)
evaluator = RecEvaluator(eval_dict, None, device)

In [5]:
writer_dis = SummaryWriter("runs/IRGAN-SGD-without-pretrained-model/Discriminator")
writer_gen = SummaryWriter("runs/IRGAN-SGD-without-pretrained-model/Generator")

In [6]:
G = Generator(num_users, num_items, emb_dim, bought_mask)
D = Discriminator(num_users, num_items, emb_dim)
G = G.to(device)
D = D.to(device)

loss_D = nn.BCELoss()
optimizer_G = torch.optim.SGD(G.parameters(), momentum = 0.9, lr = eta_G)
optimizer_D = torch.optim.SGD(D.parameters(), momentum = 0.9, lr = eta_D, weight_decay = weight_decay_d)

In [7]:
for epoch in range(epochs):
    print(
            "[Epoch %d/%d]"
            % (epoch+1, epochs)
        )  
    time_epoch_start = time.time()
    
    # ---------------------
    #  Train Discriminator
    # ---------------------   
    D.train()
    G.eval()
    for epoch_d in range(epochs_d):
        time_d_start = time.time()
        if(epoch_d % 5 == 0):
            train_set_d = dp.prepare_data_for_discriminator(G, train_ui, batch_size = batch_size)
        loss_d_epoch = 0
        for users, items, labels in train_set_d:
            dis_score = D(users,items)
            loss_d = loss_D(dis_score, labels)
            loss_d_epoch += loss_d.item()
            optimizer_D.zero_grad()
            loss_d.backward()
            optimizer_D.step()
            
        time_d_end = time.time()
        loss_d_epoch /= len(train_set_d)
        print(
            "\t[Discriminator][Epochs %d/%d] [D epoch loss: %6.5f] [Time:%6.5f] "
            % (epoch_d+1, epochs_d, loss_d_epoch, time_d_end - time_d_start)
        )
        writer_dis.add_scalar('Loss/Discriminator', loss_d_epoch, epoch*epochs_d + epoch_d)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(D, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_dis.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MAP', map10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MRR', mrr10, epoch*epochs_d + epoch_d)
        
    # --------------------- 
    #  Train Generator
    #
    # For generator
    #   Generate K user-item pairs
    #   Leveraging Policy Gradient to update parameters of generator
    #
    # --------------------- 
    D.eval()
    G.train()
    for epoch_g in range(epochs_g):
        time_g_start = time.time()
        train_set_g = dp.prepare_data_for_generator(all_users, batch_size)
        loss_g_epoch = 0
        for fake_users, in train_set_g:
            fake_items, fake_probs, fake_p_n = G.sample_items_for_users(fake_users, k = 256,\
                                                                        temperature=1, lambda_bought=0.2)
            fake_users = fake_users.view(-1,1).expand_as(fake_items).contiguous()
            fake_users = fake_users.view(-1)
            fake_items = fake_items.view(-1)
            fake_probs = fake_probs.view(-1)
            fake_p_n = fake_p_n.view(-1)
            
            log_fake_probs = torch.log(fake_probs.clamp(1e-8))
            fake_probs = fake_probs.detach()
            fake_p_n = fake_p_n.detach()
            
            reward = (2 * D(fake_users, fake_items) - 1).detach()*(fake_probs/fake_p_n)
            loss_g = -torch.mean(log_fake_probs*reward)
            loss_g_epoch += loss_g
            
            optimizer_G.zero_grad()  
            loss_g.backward() 
            optimizer_G.step()
        time_g_end = time.time()
        print(
            "\t[Generator][Epochs %d/%d] [G epoch loss: %6.5f] [Time:%6.5f]"
            % (epoch_g + 1, epochs_g, loss_g_epoch, time_g_end - time_g_start )
        ) 
        writer_gen.add_scalar('Loss/Generator', loss_g_epoch, epoch*epochs_g + epoch_g)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(G, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_gen.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MAP', map10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MRR', mrr10, epoch*epochs_g + epoch_g)
    time_epoch_end = time.time()
writer_dis.close()    
writer_gen.close()

[Epoch 1/200]
	[Discriminator][Epochs 1/100] [D epoch loss: 0.69234] [Time:8.77246] 
	[Discriminator][Epochs 2/100] [D epoch loss: 0.69064] [Time:8.40830] 
	[Discriminator][Epochs 3/100] [D epoch loss: 0.68896] [Time:8.50414] 
	[Discriminator][Epochs 4/100] [D epoch loss: 0.68730] [Time:8.63843] 
	[Discriminator][Epochs 5/100] [D epoch loss: 0.68565] [Time:7.77593] 
	[Discriminator][Epochs 6/100] [D epoch loss: 0.68402] [Time:7.78609] 
	[Discriminator][Epochs 7/100] [D epoch loss: 0.68240] [Time:7.92326] 
	[Discriminator][Epochs 8/100] [D epoch loss: 0.68080] [Time:8.12737] 
	[Discriminator][Epochs 9/100] [D epoch loss: 0.67922] [Time:10.46622] 
	[Discriminator][Epochs 10/100] [D epoch loss: 0.67765] [Time:10.40237] 
	[Discriminator][Epochs 11/100] [D epoch loss: 0.67610] [Time:10.02066] 
	[Discriminator][Epochs 12/100] [D epoch loss: 0.67456] [Time:9.88742] 
	[Discriminator][Epochs 13/100] [D epoch loss: 0.67303] [Time:10.57633] 
	[Discriminator][Epochs 14/100] [D epoch loss: 0.67152]

	[Generator][Epochs 15/50] [G epoch loss: -18.88421] [Time:0.03653]
	[Generator][Epochs 16/50] [G epoch loss: -18.93554] [Time:0.03677]
	[Generator][Epochs 17/50] [G epoch loss: -19.05986] [Time:0.03823]
	[Generator][Epochs 18/50] [G epoch loss: -19.03792] [Time:0.03842]
	[Generator][Epochs 19/50] [G epoch loss: -18.92602] [Time:0.03700]
	[Generator][Epochs 20/50] [G epoch loss: -18.89982] [Time:0.04049]
	[Generator][Epochs 21/50] [G epoch loss: -18.79659] [Time:0.03261]
	[Generator][Epochs 22/50] [G epoch loss: -18.81802] [Time:0.03808]
	[Generator][Epochs 23/50] [G epoch loss: -18.76823] [Time:0.03849]
	[Generator][Epochs 24/50] [G epoch loss: -18.83399] [Time:0.03824]
	[Generator][Epochs 25/50] [G epoch loss: -18.78799] [Time:0.03552]
	[Generator][Epochs 26/50] [G epoch loss: -18.89440] [Time:0.03734]
	[Generator][Epochs 27/50] [G epoch loss: -18.77968] [Time:0.04569]
	[Generator][Epochs 28/50] [G epoch loss: -18.76377] [Time:0.03766]
	[Generator][Epochs 29/50] [G epoch loss: -18.81

	[Discriminator][Epochs 81/100] [D epoch loss: 0.53954] [Time:9.50697] 
	[Discriminator][Epochs 82/100] [D epoch loss: 0.53911] [Time:9.60854] 
	[Discriminator][Epochs 83/100] [D epoch loss: 0.53870] [Time:9.31950] 
	[Discriminator][Epochs 84/100] [D epoch loss: 0.53828] [Time:8.87657] 
	[Discriminator][Epochs 85/100] [D epoch loss: 0.53787] [Time:10.64339] 
	[Discriminator][Epochs 86/100] [D epoch loss: 0.53746] [Time:10.49367] 
	[Discriminator][Epochs 87/100] [D epoch loss: 0.53705] [Time:9.51685] 
	[Discriminator][Epochs 88/100] [D epoch loss: 0.53665] [Time:9.03326] 
	[Discriminator][Epochs 89/100] [D epoch loss: 0.53625] [Time:9.36092] 
	[Discriminator][Epochs 90/100] [D epoch loss: 0.53585] [Time:10.66793] 
	[Discriminator][Epochs 91/100] [D epoch loss: 0.53555] [Time:10.41996] 
	[Discriminator][Epochs 92/100] [D epoch loss: 0.53516] [Time:10.10507] 
	[Discriminator][Epochs 93/100] [D epoch loss: 0.53477] [Time:10.07844] 
	[Discriminator][Epochs 94/100] [D epoch loss: 0.53438] [T

	[Discriminator][Epochs 48/100] [D epoch loss: 0.52584] [Time:11.29611] 
	[Discriminator][Epochs 49/100] [D epoch loss: 0.52557] [Time:10.39571] 
	[Discriminator][Epochs 50/100] [D epoch loss: 0.52530] [Time:10.36556] 
	[Discriminator][Epochs 51/100] [D epoch loss: 0.52501] [Time:10.60673] 
	[Discriminator][Epochs 52/100] [D epoch loss: 0.52474] [Time:9.28472] 
	[Discriminator][Epochs 53/100] [D epoch loss: 0.52447] [Time:9.34826] 
	[Discriminator][Epochs 54/100] [D epoch loss: 0.52421] [Time:10.33061] 
	[Discriminator][Epochs 55/100] [D epoch loss: 0.52395] [Time:9.26771] 
	[Discriminator][Epochs 56/100] [D epoch loss: 0.52371] [Time:9.29191] 
	[Discriminator][Epochs 57/100] [D epoch loss: 0.52345] [Time:10.57822] 
	[Discriminator][Epochs 58/100] [D epoch loss: 0.52319] [Time:10.20780] 
	[Discriminator][Epochs 59/100] [D epoch loss: 0.52293] [Time:10.47094] 
	[Discriminator][Epochs 60/100] [D epoch loss: 0.52268] [Time:10.24625] 
	[Discriminator][Epochs 61/100] [D epoch loss: 0.52239]

	[Discriminator][Epochs 15/100] [D epoch loss: 0.52123] [Time:9.37375] 
	[Discriminator][Epochs 16/100] [D epoch loss: 0.52104] [Time:9.50660] 
	[Discriminator][Epochs 17/100] [D epoch loss: 0.52085] [Time:10.40618] 
	[Discriminator][Epochs 18/100] [D epoch loss: 0.52065] [Time:10.56318] 
	[Discriminator][Epochs 19/100] [D epoch loss: 0.52046] [Time:10.18803] 
	[Discriminator][Epochs 20/100] [D epoch loss: 0.52027] [Time:10.49830] 
	[Discriminator][Epochs 21/100] [D epoch loss: 0.52006] [Time:10.52884] 
	[Discriminator][Epochs 22/100] [D epoch loss: 0.51987] [Time:9.54474] 
	[Discriminator][Epochs 23/100] [D epoch loss: 0.51968] [Time:8.22955] 
	[Discriminator][Epochs 24/100] [D epoch loss: 0.51950] [Time:8.97188] 
	[Discriminator][Epochs 25/100] [D epoch loss: 0.51931] [Time:10.12600] 
	[Discriminator][Epochs 26/100] [D epoch loss: 0.51906] [Time:10.11170] 
	[Discriminator][Epochs 27/100] [D epoch loss: 0.51887] [Time:10.48547] 
	[Discriminator][Epochs 28/100] [D epoch loss: 0.51869] 

...

	[Discriminator][Epochs 28/100] [D epoch loss: 0.41023] [Time:5.63500] 
	[Discriminator][Epochs 29/100] [D epoch loss: 0.41019] [Time:5.44813] 
	[Discriminator][Epochs 30/100] [D epoch loss: 0.41014] [Time:5.49405] 
	[Discriminator][Epochs 31/100] [D epoch loss: 0.40996] [Time:5.84462] 
	[Discriminator][Epochs 32/100] [D epoch loss: 0.40992] [Time:5.60752] 
	[Discriminator][Epochs 33/100] [D epoch loss: 0.40988] [Time:5.70809] 
	[Discriminator][Epochs 34/100] [D epoch loss: 0.40983] [Time:5.79479] 
	[Discriminator][Epochs 35/100] [D epoch loss: 0.40979] [Time:5.63321] 
	[Discriminator][Epochs 36/100] [D epoch loss: 0.40982] [Time:5.38050] 
	[Discriminator][Epochs 37/100] [D epoch loss: 0.40978] [Time:5.55614] 
	[Discriminator][Epochs 38/100] [D epoch loss: 0.40974] [Time:5.14781] 
	[Discriminator][Epochs 39/100] [D epoch loss: 0.40970] [Time:5.11102] 
	[Discriminator][Epochs 40/100] [D epoch loss: 0.40966] [Time:5.18119] 
	[Discriminator][Epochs 41/100] [D epoch loss: 0.40938] [Time:4.

	[Generator][Epochs 45/50] [G epoch loss: -65.12581] [Time:0.03981]
	[Generator][Epochs 46/50] [G epoch loss: -65.06186] [Time:0.03872]
	[Generator][Epochs 47/50] [G epoch loss: -64.84999] [Time:0.03952]
	[Generator][Epochs 48/50] [G epoch loss: -64.92279] [Time:0.03967]
	[Generator][Epochs 49/50] [G epoch loss: -65.03846] [Time:0.03903]
	[Generator][Epochs 50/50] [G epoch loss: -65.02199] [Time:0.03918]
[Epoch 198/200]
	[Discriminator][Epochs 1/100] [D epoch loss: 0.41258] [Time:5.50197] 
	[Discriminator][Epochs 2/100] [D epoch loss: 0.41251] [Time:5.51915] 
	[Discriminator][Epochs 3/100] [D epoch loss: 0.41244] [Time:5.53265] 
	[Discriminator][Epochs 4/100] [D epoch loss: 0.41238] [Time:5.51487] 
	[Discriminator][Epochs 5/100] [D epoch loss: 0.41231] [Time:5.50800] 
	[Discriminator][Epochs 6/100] [D epoch loss: 0.41204] [Time:5.45797] 
	[Discriminator][Epochs 7/100] [D epoch loss: 0.41198] [Time:5.44204] 
	[Discriminator][Epochs 8/100] [D epoch loss: 0.41191] [Time:5.55996] 
	[Discri

	[Generator][Epochs 10/50] [G epoch loss: -65.36525] [Time:0.03700]
	[Generator][Epochs 11/50] [G epoch loss: -65.03090] [Time:0.03734]
	[Generator][Epochs 12/50] [G epoch loss: -65.51977] [Time:0.03732]
	[Generator][Epochs 13/50] [G epoch loss: -65.47314] [Time:0.03644]
	[Generator][Epochs 14/50] [G epoch loss: -65.32663] [Time:0.03707]
	[Generator][Epochs 15/50] [G epoch loss: -65.17911] [Time:0.03514]
	[Generator][Epochs 16/50] [G epoch loss: -65.23911] [Time:0.03677]
	[Generator][Epochs 17/50] [G epoch loss: -65.04155] [Time:0.03820]
	[Generator][Epochs 18/50] [G epoch loss: -65.07352] [Time:0.03718]
	[Generator][Epochs 19/50] [G epoch loss: -65.08113] [Time:0.03700]
	[Generator][Epochs 20/50] [G epoch loss: -65.00632] [Time:0.03740]
	[Generator][Epochs 21/50] [G epoch loss: -64.98347] [Time:0.03714]
	[Generator][Epochs 22/50] [G epoch loss: -65.30060] [Time:0.03696]
	[Generator][Epochs 23/50] [G epoch loss: -65.18311] [Time:0.03666]
	[Generator][Epochs 24/50] [G epoch loss: -65.21

	[Discriminator][Epochs 76/100] [D epoch loss: 0.40879] [Time:6.35687] 
	[Discriminator][Epochs 77/100] [D epoch loss: 0.40875] [Time:5.95300] 
	[Discriminator][Epochs 78/100] [D epoch loss: 0.40872] [Time:5.52163] 
	[Discriminator][Epochs 79/100] [D epoch loss: 0.40869] [Time:5.14548] 
	[Discriminator][Epochs 80/100] [D epoch loss: 0.40866] [Time:5.18545] 
	[Discriminator][Epochs 81/100] [D epoch loss: 0.40876] [Time:5.25570] 
	[Discriminator][Epochs 82/100] [D epoch loss: 0.40873] [Time:5.13208] 
	[Discriminator][Epochs 83/100] [D epoch loss: 0.40870] [Time:4.78793] 
	[Discriminator][Epochs 84/100] [D epoch loss: 0.40867] [Time:5.17062] 
	[Discriminator][Epochs 85/100] [D epoch loss: 0.40864] [Time:5.15456] 
	[Discriminator][Epochs 86/100] [D epoch loss: 0.40856] [Time:5.13704] 
	[Discriminator][Epochs 87/100] [D epoch loss: 0.40853] [Time:5.16193] 
	[Discriminator][Epochs 88/100] [D epoch loss: 0.40850] [Time:5.17368] 
	[Discriminator][Epochs 89/100] [D epoch loss: 0.40847] [Time:5.

	[Discriminator][Epochs 43/100] [D epoch loss: 0.40956] [Time:5.17209] 
	[Discriminator][Epochs 44/100] [D epoch loss: 0.40953] [Time:5.18089] 
	[Discriminator][Epochs 45/100] [D epoch loss: 0.40949] [Time:5.21733] 
	[Discriminator][Epochs 46/100] [D epoch loss: 0.40984] [Time:5.22580] 
	[Discriminator][Epochs 47/100] [D epoch loss: 0.40981] [Time:5.28586] 
	[Discriminator][Epochs 48/100] [D epoch loss: 0.40977] [Time:5.23012] 
	[Discriminator][Epochs 49/100] [D epoch loss: 0.40974] [Time:5.26826] 
	[Discriminator][Epochs 50/100] [D epoch loss: 0.40970] [Time:5.25444] 
	[Discriminator][Epochs 51/100] [D epoch loss: 0.40954] [Time:5.22707] 
	[Discriminator][Epochs 52/100] [D epoch loss: 0.40950] [Time:5.19227] 
	[Discriminator][Epochs 53/100] [D epoch loss: 0.40947] [Time:5.10598] 
	[Discriminator][Epochs 54/100] [D epoch loss: 0.40944] [Time:5.11561] 
	[Discriminator][Epochs 55/100] [D epoch loss: 0.40940] [Time:5.19571] 
	[Discriminator][Epochs 56/100] [D epoch loss: 0.40962] [Time:5.